In [1]:
import pandas as pd
import numpy as np
import glob
import sys
from datetime import datetime,timedelta

In [2]:
folder="/home/jian/Projects/MOTV/DMP data/Oct/"
file_list=[]
for file_name in glob.glob(folder+'*.csv'):
    x = pd.read_csv(file_name,encoding="ISO-8859-1")
    x.reset_index(inplace=True)
    x['real_time']=x['real_time'].astype(str)
    file_list.append(x)

data=pd.DataFrame()
for df in file_list:
    for i in df['index']:
        if df['real_time'][i]=='nan':
            if df['viewer_id'][i][(len(df['viewer_id'][i])-1):len(df['viewer_id'][i])]=='"':
                df['viewer_id'][i]=df['viewer_id'][i][0:(len(df['viewer_id'][i])-1)]
            if df['custom_pub_id3'][i-1][(len(df['custom_pub_id3'][i-1])-1):len(df['custom_pub_id3'][i-1])]=='\r':
                df['custom_pub_id3'][i-1]=df['custom_pub_id3'][i-1][0:(len(df['custom_pub_id3'][i-1])-1)]
            df['custom_pub_id3'][i-1]=df['custom_pub_id3'][i-1]+df['viewer_id'][i]
    df=df[df['real_time']!="nan"]    
    data=data.append(df)
data.reset_index(inplace=True)
del data['index']
data.to_csv("/home/jian/Projects/MOTV/Data in One/From_Oct5_to_Dec_2.csv")

del data['display_width']
del data['display_height']


data.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(529583, 29)

In [5]:
def adjust_timezone(x, y):
     return x + timedelta(hours=y)

In [6]:
data['time_zone'].fillna(value=0,inplace=True)
data['time']= data['hour_time'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M'))
data['time_adjusted']=data.apply(lambda x: adjust_timezone(x['time'],x['time_zone']),axis=1)
data['day_of_week']= data['time_adjusted'].dt.dayofweek
data['hour']= data['time_adjusted'].dt.hour

data['weekday']=np.where(data['day_of_week']==0,'Monday',
                         np.where(data['day_of_week']==1,'Tuesday',
                                  np.where(data['day_of_week']==2,'Wednesday',
                                           np.where(data['day_of_week']==3,'Thursday',
                                                    np.where(data['day_of_week']==4,'Friday',
                                                             np.where(data['day_of_week']==5,'Saturday',
                                                                      np.where(data['day_of_week']==6,'Sunday','error'
                                 )))))))

In [7]:
data.columns

Index(['level_0', 'viewer_id', 'hour_time', 'real_time', 'domain', 'home_page',
       'ip', 'time_zone', 'video_position', 'device_type', 'protocol',
       'app_type', 'is_secured', 'isp', 'continent', 'Language', 'country',
       'state', 'city', 'zipcode', 'latitude', 'longitude', 'OS', 'os_version',
       'device_model_name', 'device_brand', 'custom_pub_id', 'custom_pub_id2',
       'custom_pub_id3', 'time', 'time_adjusted', 'day_of_week', 'hour',
       'weekday'],
      dtype='object')

In [37]:
Data_Video=data[data['domain']=="com.myoutdoortv"]
Data_Video.shape

Data_Action=Data_Video.loc[:,['viewer_id','real_time','custom_pub_id','custom_pub_id2','custom_pub_id3']]
Data_Action.reset_index(inplace=True)
del Data_Action['index']

Data_Action['actions']=Data_Action['custom_pub_id'].apply(lambda x: x.split(':')[0])
Data_Action=Data_Action[Data_Action['actions'].isin(['start_video','leave_video'])]
Data_Action.reset_index(inplace=True)
del Data_Action['index']
Data_Action['custom_pub_id2']=Data_Action['custom_pub_id2'].astype(str)
Data_Action['custom_pub_id3']=Data_Action['custom_pub_id3'].astype(str)

Data_Action['custom_pub_id2']=np.where(Data_Action['custom_pub_id2']=='nan','||',Data_Action['custom_pub_id2'])
Data_Action['custom_pub_id3']=np.where(Data_Action['custom_pub_id3']=='nan','||',Data_Action['custom_pub_id3'])

Data_Action['ShowID']=Data_Action['custom_pub_id2'].apply(lambda x: x.split('|')[0])
Data_Action['ShowName']=Data_Action['custom_pub_id2'].apply(lambda x: x.split('|')[1])
Data_Action['VideoID']=Data_Action['custom_pub_id2'].apply(lambda x: x.split('|')[2])

Pub_ID3_Split=Data_Action['custom_pub_id3'].apply(lambda x: x.split('|'))

Data_Action=Data_Action.loc[Pub_ID3_Split.apply(lambda x: len(x))==3,] # remove 5 records in Oct 5 about 162989 Evil Forces|Uncle Ted explains h

Data_Action['VideoName']=Data_Action['custom_pub_id3'].apply(lambda x: x.split('|')[0])
Data_Action['video_duration']=Data_Action['custom_pub_id3'].apply(lambda x: x.split('|')[1])
Data_Action['position']=Data_Action['custom_pub_id3'].apply(lambda x: x.split('|')[2])
Data_Action.drop_duplicates(inplace=True)


In [39]:
Video_Viewship=pd.merge(Data_Action_with_ID_Start,Data_Action_with_ID_Leave,on=["viewer_id","VideoID"],how="inner")


Video_Viewship['view_length']=Video_Viewship['leave_position']-Video_Viewship['start_position']
Video_Viewship['mean_view_length']=Video_Viewship['view_length'].groupby(Video_Viewship['VideoID']).transform('mean')

Video_Viewship_Test=Video_Viewship[['VideoID','mean_view_length']]
Video_Viewship_Test.drop_duplicates(inplace=True)
# Video_Viewship_Test.reset_index(inplace=True)
# del Video_Viewship_Test['index']

Video_duration=Data_Action[["VideoID","video_duration"]]
Video_duration.drop_duplicates(inplace=True)
# Video_duration.reset_index(inplace=True)
# del Video_duration['index']
Video_duration['video_duration']=Video_duration['video_duration'].astype(str)

Video_duration['str_len_org']=Video_duration['video_duration'].apply(lambda x:len(x))
Video_duration=Video_duration[Video_duration['video_duration']!='nan']
Video_duration.reset_index(inplace=True)
del Video_duration['index']
Video_duration['video_duration']=Video_duration['video_duration'].str.strip()
Video_duration['video_duration']=np.where(Video_duration['video_duration'].str.len()==5,str('00:')+Video_duration['video_duration'],Video_duration['video_duration'])
Video_duration['video_duration']=np.where(Video_duration['video_duration'].str.len()==1,
                                          str(Video_duration['video_duration'])[0:2]+str(":")+str(Video_duration['video_duration'])[3:5]+str(":")+str(Video_duration['video_duration'])[6:8],
                                          Video_duration['video_duration'])

    
        
Video_duration['video_duration']=Video_duration['video_duration'].str.replace('.', ':')
Test=Video_duration['video_duration'].apply(lambda x: x.split(':'))
Test=pd.DataFrame(Test)
Video_duration['FirstSplit']=Test['video_duration'].apply(lambda x:int(x[0]))
Video_duration['SecondSplit']=Test['video_duration'].apply(lambda x:int(x[1]))
Video_duration['ThirdSplit']=Test['video_duration'].apply(lambda x:x[2])
Video_duration['ThirdSplit']=Video_duration['ThirdSplit'].str.replace('@', '2') # Correct @ into 2 for 42064|Big Bass Battle|44354 Okeechobee Part|00:22:@2|
Video_duration['ThirdSplit']=Video_duration['ThirdSplit'].astype(int)
Video_duration['Video_Duration_In_Second']=Video_duration['FirstSplit']*3600+Video_duration['SecondSplit']*60+Video_duration['ThirdSplit']
Video_duration=Video_duration[['VideoID','video_duration','Video_Duration_In_Second']]

/Users/JayLiang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/JayLiang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/JayLiang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/JayLiang/

In [31]:
Video_Viewship=pd.merge(Video_Viewship,Video_duration,on='VideoID',how='left')
Video_Viewship=pd.merge(Video_Viewship,New_mapping,on="VideoID",how='left')
Video_Viewship_Roll_to_Video=Video_Viewship[['VideoID','mean_view_length','Video_Duration_In_Second']]
Video_Viewship_Roll_to_Video.drop_duplicates(inplace=True)
Video_Viewship_Roll_to_Video.reset_index(inplace=True)
del Video_Viewship_Roll_to_Video['index']
Video_Viewship_Roll_to_Video['Video_View_Pctg']=Video_Viewship_Roll_to_Video['mean_view_length']/Video_Viewship_Roll_to_Video['Video_Duration_In_Second']


Video_Viewship_Roll_to_Show=Video_Viewship[["VideoID",'ShowID','view_length','Video_Duration_In_Second']]
Video_Viewship_Roll_to_Show=Video_Viewship_Roll_to_Show.drop_duplicates()
Video_Viewship_Roll_to_Show.reset_index(inplace=True)
del Video_Viewship_Roll_to_Show['index']
Video_Viewship_Roll_to_Show_Viewed=Video_Viewship_Roll_to_Show.groupby(['ShowID'])[['view_length','Video_Duration_In_Second']].sum()
Video_Viewship_Roll_to_Show_Viewed.reset_index(inplace=True)
Video_Viewship_Roll_to_Show_Viewed['Show_View_Pctg']=Video_Viewship_Roll_to_Show_Viewed['view_length']/Video_Viewship_Roll_to_Show_Viewed['Video_Duration_In_Second']

Match_Table_All_Show=New_mapping[['ShowID','ShowName']]
Match_Table_All_View=New_mapping[['VideoID','VideoName']]
Match_Table_All_View['VideoID']=Match_Table_All_View['VideoID'].astype(str)
Match_Table_All_Show=Match_Table_All_Show[~pd.isnull(Match_Table_All_Show['ShowID'])]
Match_Table_All_Show['ShowID']=Match_Table_All_Show['ShowID'].astype(int).astype(str)
Start_Video_Table_Show.drop_duplicates(inplace=True)


Start_Video_Table_Show=pd.merge(Start_Video_Table_Show,Match_Table_All_Show,on='ShowID',how='left')
Start_Video_Table_Show.drop_duplicates(inplace=True)
Start_Video_Table_Show.reset_index(inplace=True)
del Start_Video_Table_Show['index']

Start_Video_Table_Video=pd.merge(Start_Video_Table_Video,Match_Table_All_View,on='VideoID',how='left')


/Users/JayLiang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/JayLiang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/JayLiang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [32]:
Video_Viewship_Roll_to_Show_Viewed['ShowID']=Video_Viewship_Roll_to_Show_Viewed['ShowID'].astype(int)
Video_Viewship_Roll_to_Show_Viewed['ShowID']=Video_Viewship_Roll_to_Show_Viewed['ShowID'].astype(str)
Show_Output=pd.merge(Start_Video_Table_Show,Video_Viewship_Roll_to_Show_Viewed,on="ShowID",how="outer")
Video_Output=pd.merge(Start_Video_Table_Video,Video_Viewship_Roll_to_Video,on="VideoID",how="outer")

writer = pd.ExcelWriter('/Users/JayLiang/Desktop/Media Storm/DMP/Output/Video/New_Mapping_Oct5_to_Oct23_Video_Viewership.xlsx')
Show_Output.to_excel(writer,"Show")
Video_Output.to_excel(writer,"Video")
writer.save()

In [79]:
'''
test_pub_id2=test['custom_pub_id2'][~pd.isnull(test['custom_pub_id2'])].apply(lambda x: str(x).split("|"))
VideoID_all=test_pub_id2.apply(lambda x:x[2]).drop_duplicates()
All_IDs=VideoID_all.to_frame(name="VideoID_all")
All_IDs.reset_index(inplace=True)
del All_IDs['index']
ShowID_all=test_pub_id2.apply(lambda x:x[0]).drop_duplicates()
temp=ShowID_all.to_frame(name="ShowID_all")
temp.reset_index(inplace=True)
All_IDs['ShowID_all']=temp['ShowID_all']
All_IDs.to_csv("/Users/JayLiang/Desktop/Media Storm/DMP/Output/All_IDs_Oct5_to_Oct23.csv")
# All_IDs=pd.DataFrame(ShowID_all,VideoID_all)
'''